In [30]:
# ------------------------------------------------------------------------------------------------------------------------ #
# ---------------------------------------- 0.import library ---------------------------------------- # 
# ------------------------------------------------------------------------------------------------------------------------ #

# module : 설치가 되어 있지 않으면 설치 필요
import sys
import os
import xlrd
import pandas as pd
import numpy as np
import statistics
import math
import datetime
import time
import re # regural expression
import functools
import random
import gc
import pickle
import warnings
import scipy
import sympy as sym

#
from sys import stdout
from functools import reduce # apply function simultaneously on all elements  
from io import StringIO
from pandas.api.types import CategoricalDtype # convert categorical data type 
from itertools import groupby
from itertools import product
from itertools import chain
from itertools import cycle, islice
from collections import defaultdict
from datetime import datetime # set datetime class 
from collections import Counter
from scipy import stats 

### plot ###
import matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns 
from matplotlib.ticker import MultipleLocator, IndexLocator, FuncFormatter
from matplotlib.dates import MonthLocator, DateFormatter
from matplotlib import gridspec

# 
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.metrics import r2_score
from scipy.signal import savgol_filter
from scipy.ndimage.filters import gaussian_filter1d # line smoothing    
from tqdm import tqdm
import xgboost


# plot에 한글 사용을 위해서 세팅 : plot에서 한글이 깨지는 경우 font를 직접 지정 해 주어야 함 
from matplotlib import font_manager, rc

# 
font_path = "C:/Windows/Fonts/malgun.ttf" # 맑은고딕
font = font_manager.FontProperties(fname = font_path).get_name()
rc('font', family = font)


# check module version
print("- python version :", sys.version)# 파이썬 버전
print("- pandas version :", pd.__version__) # pandas 버전
print("- seaborn version :", sns.__version__) # seaborn 버전
print("- matplotlib version :", matplotlib.__version__) # matplotlib 버전


# set options
pd.set_option('display.max_row', 100) # row 100개까지 display 
pd.set_option('display.max_columns', 120) # column 120개까지 display

# set work directory
# os.chdir(r"C:\Users\2019002\Desktop\전기로국책과제")

# 
print("- dir path :", os.getcwd())

# ignore warnings
warnings.filterwarnings("ignore")
# ------------------------------------------------------------------------------------------------------------------------ #

- python version : 3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]
- pandas version : 1.4.4
- seaborn version : 0.11.2
- matplotlib version : 3.5.2
- dir path : \\10.216.153.164\home\begas\27. AI활용 전기로 에너지 효율 최적화 기술개발\DAT\rawdata\2차년도


In [31]:
# percentile for aggregate summary 

def trim_mean(n):
    def trim_mean_(x):
        return stats.trim_mean(x, n)
    ;
    
    trim_mean_.__name__ = 'trim_mean_%s' % n
    return trim_mean_
;


def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    ;
    
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_
;

# calculate time delta 
def fn_tmp_timesec(data) :

    # sorting 
    data = data.sort_values("Time", ascending = True)

    # 
    total_prc_tm = (max(data.Time) - min(data.Time)).total_seconds()
    data["Timedelta"] = [max(data.Time) - x for x in data["Time"]]
    data["Timedsec"] = [total_prc_tm - x.total_seconds() for x in data["Timedelta"]]
    data["Time_minute"] = [x/60 for x in data["Timedsec"]]
    
    retrun_dt = data[df_cols + ["Timedelta", "Timedsec", "Time_minute"]] 
    return retrun_dt
;


''

###  Data 불러오기

In [32]:
# 2023년 장입량 데이터 : 2023년 월단위로 별도로 추가하여 갱신 필요
# xls 파일로 작업된 내용은 원본을 토대로 기존 항목에 맞춰 동일하게 항목명을 통일시킨 csv파일 활용함
# os.chdir(r"D:\국책과제_2차년도\DAT\rawdata\2차년도\charge장입량_year23_csv\mes")
os.chdir(r"\\10.216.153.164\home\begas\27. AI활용 전기로 에너지 효율 최적화 기술개발\DAT\rawdata\2차년도\charge장입량_year23_csv\mes")
chr_folderlist = os.listdir()

chr_file_ls = []
for chr_file in chr_folderlist : 

    print(chr_file)
    # import file
    chr_file_dt = pd.read_csv(chr_file, encoding = "cp949")
    chr_file_ls.append(chr_file_dt)
;

# 
input_ton_charge_2023 = pd.concat(chr_file_ls, axis = 0)
input_ton_charge_2023.sort_values("heat", inplace = True)

# 
input_ton_charge_2023 = input_ton_charge_2023[~pd.isna(input_ton_charge_2023["heat"])]
input_ton_charge_2023["heat"] = ["E" + str(int(x)) for x in input_ton_charge_2023["heat"]]
input_ton_charge_2023.rename(columns = {"heat" : "HEAT_NO"}, inplace = True)

# 
reshape_input_ton_charge_2023 = input_ton_charge_2023[["HEAT_NO", "1cha_ton", "2cha_ton", "3cha_ton"]].melt(id_vars = ["HEAT_NO"]) 
reshape_input_ton_charge_2023["ActualCHRG_PX3"] = [int(x[0]) for x in reshape_input_ton_charge_2023["variable"]]
reshape_input_ton_charge_2023.rename(columns = {"value" : "MES_ton"}, inplace = True)
reshape_input_ton_charge_2023["MES_ton"] = reshape_input_ton_charge_2023["MES_ton"].fillna(0)
reshape_input_ton_charge_2023["tot_mes_ton"] = reshape_input_ton_charge_2023.groupby(["HEAT_NO"])["MES_ton"].transform(lambda x: sum(x))
reshape_input_ton_charge_2023["장입횟수"] = reshape_input_ton_charge_2023.groupby(["HEAT_NO"])["MES_ton"].transform(lambda x : len(x) - sum(x ==0))

Apr_mes.csv
Aug_mes.csv
Dec_mes.csv
Feb_mes.csv
Jan_mes.csv
Jul_mes.csv
Jun_mes.csv
Mar_mes.csv
May_mes.csv
Nov_mes.csv
Oct_mes.csv
Sep_mes.csv


In [33]:
col_select = ["Month", "HEAT_NO", "SmartArcNumberOV", "Time", "ActualCHRG_PX3", "ActFceTap_Px3", "Charge_MWH", "MWH_PX3",  
              "SF1", "SF2", "SF3", "VoltSF1", "VoltSF2", "VoltSF3", "ExtSmartFactor1", "ExtSmartFactor2", "ExtSmartFactor3",
              'Current1', 'Current2', 'Current3', 'Voltage1', 'Voltage2', 'Voltage3',
              "ZFeedback1", "ZFeedback2", "ZFeedback3", "Res1", "Res2", "Res3", 
              "Reactance1", "Reactance2", "Reactance3", "ArcLength1", "ArcLength2", "ArcLength3",
              "MVA1", "MVA2", "MVA3", "MVATot", "MWTot", "MVARTot", "PFTot", 
              "MW1", "MW2", "MW3", "MVAR1", "MVAR2", "MVAR3", "PF1", "PF2", "PF3", "RWI1", "RWI2", "RWI3", 
              "ArcEfficiency1", "ArcEfficiency2", "ArcEfficiency3", "I2H1", "I2H2", "I2H3", 
              "VoltageLineToLine12", "VoltageLineToLine23", "VoltageLineToLine31",
              "TimeSincePwrOn", "PX3SteelTemp",
              "PX3BurnerO2", "PX3LanceO2", 
              "Px3ChargeCarbon", "PX3InjecCarbon", "All-mixConsumption",
              "PX3Lime", "PX3DoloLime",
              "B5_CarbonFlow", "B1_CarbonFlow", "CarbTopFeed_Cons", 
              "B5_O2MainFlow", "B1_O2MainFlow", "B2_O2MainFlow", "B4_O2MainFlow", 
              "B1_O2ShroudFlow", "B2_O2ShroudFlow", "B4_O2ShroudFlow",
              "SFCE_CO2_Pct", "SFCE_CO_Pct", "SFCE_H2O_Pct", "SFCE_Offgas_MassFlow",
              "NormH2OPctFlow", "NormCO2PctFlow", "NormCOPctFlow", "COCO2Ratio_Clus"]

In [22]:
os.chdir(r"\\10.216.153.164\home\begas\27. AI활용 전기로 에너지 효율 최적화 기술개발\DAT\rawdata\2차년도")
# 2023년 상반기 데이터 
arc_2023H1_SNOV_9_filter = pd.read_pickle(r"arc_2023_SNOV_9_filter_2023H1.pkl")

# 2023년 하반기 데이터 
arc_2023H2_SNOV_9_filter = pd.read_pickle(r"arc_2023_SNOV_9_filter_2023H2.pkl")

In [23]:
# 
arc_2023H1_SNOV_9_filter.reset_index(drop = True, inplace = True)
arc_2023H2_SNOV_9_filter.reset_index(drop = True, inplace = True)

In [24]:
#
arc_2023_analysis = pd.concat([arc_2023H1_SNOV_9_filter[col_select + ["total_time"]], 
                               arc_2023H2_SNOV_9_filter[col_select + ["total_time"]]], 
                               axis = 0, ignore_index = True)

# Post Consumtion Factor
arc_2023_analysis["Year"] = 2023
arc_2023_analysis["PCFactor"] = arc_2023_analysis["NormCO2PctFlow"]/(arc_2023_analysis["NormCOPctFlow"] + arc_2023_analysis["NormCO2PctFlow"])

In [34]:
# charge 별 총 전력사용량 (Charge 3 승온기 포함)
# charge 별 전력사용량 MWH_PX3 항목 max 값 이용하여 계산하는게 정확하므로 수정 필요
arc_2023_analysis["input_chr_MWH"] = arc_2023_analysis.groupby(["HEAT_NO", "ActualCHRG_PX3"])["Charge_MWH"].transform(lambda x : max(x) - min(x))

In [35]:
# 2023 copy : charge 장입량
reshape_input_ton_charge = reshape_input_ton_charge_2023.copy()

In [36]:
# 장입량, 전력량 Table : 2023
KWH_T_table = pd.merge(arc_2023_analysis[["Month", "HEAT_NO", "ActualCHRG_PX3", "input_chr_MWH"]].drop_duplicates(),
                       reshape_input_ton_charge[["HEAT_NO", "ActualCHRG_PX3", "MES_ton", "tot_mes_ton"]].drop_duplicates(), 
                       on = ["HEAT_NO", "ActualCHRG_PX3"])
KWH_T_table.head()

# charge 별 장입전력원단위 
KWH_T_table["KWH_T"] = 1000*KWH_T_table["input_chr_MWH"]/KWH_T_table["MES_ton"]
KWH_T_table

,Month,HEAT_NO,ActualCHRG_PX3,input_chr_MWH,MES_ton,tot_mes_ton,KWH_T
0,1,E235375,1,15.900024,51.7,97.8,307.543993
1,1,E235375,2,8.103149,32.0,97.8,253.223419
2,1,E235375,3,11.696899,14.1,97.8,829.567334
3,1,E235377,1,16.696899,46.9,95.5,356.010648
4,1,E235377,2,6.501587,31.5,95.5,206.399585
...,...,...,...,...,...,...,...
14799,12,E240725,2,6.599243,31.0,95.0,212.878812
14800,12,E240725,3,12.400024,15.0,95.0,826.668294
14801,12,E240726,1,14.798462,47.0,96.0,314.860892
14802,12,E240726,2,6.798462,30.0,96.0,226.615397


In [37]:
pd.unique(arc_2023_analysis["Month"])

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12], dtype=int64)

In [38]:
fulldata = arc_2023_analysis.copy()

In [39]:
# charge 별 조업시간 60분 이상 case HEAT 제외 
longterm_heats = fulldata[fulldata["total_time"] > 60]["HEAT_NO"].unique().tolist()
fulldata = fulldata[~fulldata["HEAT_NO"].isin(longterm_heats)]

In [43]:
# HEAT별 TAP 전압구간 ID 지정 
# 전압변동 확인 
fulldata["ActFceTap_diff"] = fulldata.groupby(["HEAT_NO", "ActualCHRG_PX3"])["ActFceTap_Px3"].transform(lambda x : (x - x.shift(1)).fillna(0))
fulldata["ActFceTap_diff"] = np.where(fulldata["ActFceTap_diff"] != 0, 1, 0)

# 
fulldata["int_gr_id"] = fulldata.groupby(["HEAT_NO", "ActualCHRG_PX3"])["ActFceTap_diff"].cumsum()

 # 1개의 HEAT 내에서 각 charge 별로 전압패턴이 변경되는 경우의 id 재설정 : rank 이용
# arc_2022H2_SNOV_9_filter["int_gr_0"] = arc_2022H2_SNOV_9_filter.groupby(["HEAT_NO", "ActualCHRG_PX3", "ActFceTap_Px3"])["int_gr_id"].transform(lambda x : x.rank(method = 'dense'))
fulldata["int_gr_0"] = fulldata["int_gr_id"]
fulldata["h_sort_id_col"] = fulldata.groupby(["HEAT_NO", "ActualCHRG_PX3", "ActFceTap_Px3"])["int_gr_id"].transform(lambda x : x.rank(method = 'dense'))


In [46]:
#
fulldata["Tap_sort_key0"] = 100000*fulldata["int_gr_id"] + 100*fulldata["int_gr_id"] + fulldata["h_sort_id_col"]
fulldata["Tap_sort_key"] = fulldata.groupby(["HEAT_NO", "ActualCHRG_PX3"])["Tap_sort_key0"].transform(lambda x : x.rank(method = 'dense', numeric_only = True))
fulldata["ActFceTap_tag"] = [str(x) + "_V" + str(int(y)) for x, y in zip(fulldata["ActFceTap_Px3"], fulldata["h_sort_id_col"])]

In [49]:
# 각 전압번호 별 속하는 HEAT의 개수 확인하기 
unique_selected = fulldata[["Month", "HEAT_NO", "ActualCHRG_PX3", "ActFceTap_Px3", "int_gr_0"]].drop_duplicates()

# 
HEAT_cnt_ActFceTap = fulldata.groupby(["Month", "ActualCHRG_PX3", "ActFceTap_Px3", "int_gr_0"]).agg(cnt_heat = ("HEAT_NO", lambda x : len(x))).reset_index()

In [50]:
# 총합 대상 항목 
target_vars_sum = ["SF1", "SF2", "SF3", 
                   "VoltSF1", "VoltSF2", "VoltSF3",
                   "ExtSmartFactor1", "ExtSmartFactor2", "ExtSmartFactor3" ]

In [52]:
# HEAT별 총합 집계 
agg_vsums_heat_stat = fulldata[["Month", "HEAT_NO", "ActualCHRG_PX3"] + target_vars_sum].groupby(["Month", "HEAT_NO", "ActualCHRG_PX3"]).agg(
    
    ["sum"]
    
).reset_index()

agg_vsums_heat_stat.columns = ["_".join(pair) for pair in agg_vsums_heat_stat.columns]
agg_vsums_heat_stat.rename(columns = {"Month_" : "Month",
                                      "HEAT_NO_" : "HEAT_NO",
                                      "ActualCHRG_PX3_" : "ActualCHRG_PX3"}, inplace = True)

#
agg_vsums_heat_stat

,Month,HEAT_NO,ActualCHRG_PX3,SF1_sum,SF2_sum,SF3_sum,VoltSF1_sum,VoltSF2_sum,VoltSF3_sum,ExtSmartFactor1_sum,ExtSmartFactor2_sum,ExtSmartFactor3_sum
0,1,E235375,1,61230.0,51204.0,54230.0,11504.0,9320.0,10400.0,281913.0,170122.0,303270.0
1,1,E235375,2,34327.0,33718.0,30480.0,6444.0,6488.0,5872.0,113323.0,80588.0,72271.0
2,1,E235375,3,42605.0,43815.0,42043.0,23056.0,22816.0,22368.0,668495.0,703094.0,701369.0
3,1,E235377,1,63898.0,58406.0,67055.0,11336.0,10968.0,11984.0,146374.0,189576.0,371246.0
4,1,E235377,2,27233.0,27986.0,27977.0,4724.0,4896.0,4776.0,92591.0,96131.0,125841.0
...,...,...,...,...,...,...,...,...,...,...,...,...
14754,12,E240725,2,26048.0,24671.0,21891.0,4588.0,4532.0,4252.0,62210.0,58516.0,82332.0
14755,12,E240725,3,45278.0,45215.0,40461.0,25344.0,25136.0,24880.0,860736.0,866063.0,791034.0
14756,12,E240726,1,50535.0,46775.0,48995.0,11112.0,10736.0,11392.0,240541.0,223583.0,303614.0
14757,12,E240726,2,28659.0,26383.0,26142.0,6524.0,6232.0,6832.0,160177.0,123192.0,129221.0


### Tap 별 TopFeedCon 투입 패턴 분석 (추가 진행 필요)

In [19]:
def fn_carbonfeed_calculate(data) :
    
    # first time 
    time_min = min(data["Time"])
    
    # Tap 구간 최초값 확인
    CarbTopFeed_min = data[data["Time"] == time_min]["CarbTopFeed_Cons"].values[0]
    CarbTopFeed_max = data[data["Time"] == max(data["Time"])]["CarbTopFeed_Cons"].values[0]
    
    # check change values 
    cfeed_list = data["CarbTopFeed_Cons"].unique().tolist()
    
    # 총 변동량 기입(carbontopfeed range) 
    input_range = max(cfeed_list) - min(cfeed_list)
    
    # Tap 내 투입시점 확인 
    change_time = data[(data["CarbTopFeed_Cons"] - data["CarbTopFeed_Cons"].shift(1)).fillna(0) != 0]["Time"].tolist()
    change_time_ls = [time_min] + change_time
    
    #  
    if len(cfeed_list) < 2 :
        
        # input count
        cnt_input = 0
        
        # 세부 변동량 
        input_diff = [0]
        
    else : 
        
        # 값의 변동이 일어남 : Tab 중간에 투입이 일어남
        # input count
        cnt_input = len(cfeed_list) - 1
        
        # 세부 변동량 : 괴탄 투입 횟수에 따라 상이함. 2회 이상의 투입이 발생한 경우는 개별 투입량 tracking
        input_diff = (pd.Series(cfeed_list) - pd.Series(cfeed_list).shift(1)).fillna(0).tolist()
        
    ;
    
    # "input_count" : ["_".join(["input_count", str(x)]) for x in range(0, len(input_diff))]
    return_dt = pd.DataFrame({"CarbTopFeed_min" : CarbTopFeed_min,
                              "CarbTopFeed_max" : CarbTopFeed_max,
                              "cnt_input" : cnt_input, 
                              "input_value" : input_diff,
                              "input_range" : input_range,
                              "input_count" : [str(x) for x in range(0, len(input_diff))],
                              "change_time" : change_time_ls})
    
    # 
    return_dt = return_dt.pivot_table(index = ["CarbTopFeed_min", "CarbTopFeed_max", "cnt_input", "input_range"],
                                      columns = "input_count", 
                                      values = ["input_value", "change_time"]).reset_index()
    return_dt.columns = ["_".join(pair) for pair in return_dt.columns]
    return_dt.rename(columns = {"CarbTopFeed_min_" : "CarbTopFeed_min",
                                "CarbTopFeed_max_" : "CarbTopFeed_max",
                                "cnt_input_" : "cnt_input",
                                "input_range_" : "input_range"}, inplace = True)
        
    return return_dt
;

''

In [20]:
# fulldata[fulldata["HEAT_NO"] == "E240017"]
# 괴탄 다회 투입되는 CASE에 대하여 PX3 항목과 비교 분석 필요함

# apply aggregate function 
agg_carbfeed_interval_stat = fulldata.groupby(["Month", "HEAT_NO", "ActualCHRG_PX3",
                                               "ActFceTap_Px3", "ActFceTap_tag", "Tap_sort_key"]).apply(fn_carbonfeed_calculate).reset_index()

# sorting 
agg_carbfeed_interval_stat.sort_values(by = ["Month", "HEAT_NO", "ActualCHRG_PX3", "Tap_sort_key"], ascending = True, inplace = True)

# refill na values
agg_carbfeed_interval_stat = agg_carbfeed_interval_stat.fillna(0)
agg_carbfeed_interval_stat

,Month,HEAT_NO,ActualCHRG_PX3,ActFceTap_Px3,ActFceTap_tag,Tap_sort_key,level_6,CarbTopFeed_min,CarbTopFeed_max,cnt_input,input_range,input_value_0,input_value_1,input_value_2,input_value_3,input_value_4,input_value_5,input_value_6,input_value_7,input_value_10,input_value_11,input_value_12,input_value_13,input_value_14,input_value_15,input_value_16,input_value_17,input_value_18,input_value_19,input_value_8,input_value_9,input_value_20,input_value_21,input_value_22,input_value_23,input_value_24,input_value_25,input_value_26,input_value_27,input_value_28,input_value_29,input_value_30,input_value_31,input_value_32,input_value_33,input_value_34,input_value_35,input_value_36,input_value_37,input_value_38,input_value_39,input_value_40,input_value_41,input_value_42,input_value_43,input_value_44,input_value_45,input_value_46,input_value_47,input_value_48,input_value_49,input_value_50,input_value_51,input_value_52,input_value_53,input_value_54,input_value_55,input_value_56,input_value_57,input_value_58,input_value_59,input_value_60,input_value_61,input_value_62,input_value_63,input_value_64,input_value_65,input_value_66,input_value_67,input_value_68,input_value_69,input_value_70,input_value_71
0,1,E235375,1,7,7_V1,1.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,E235375,1,8,8_V1,2.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,E235375,1,9,9_V1,3.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,E235375,1,10,10_V1,4.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1,E235375,1,11,11_V1,5.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120047,12,E240726,3,10,10_V2,5.0,0,513.0,513.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
120045,12,E240726,3,9,9_V2,6.0,0,513.0,513.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
120043,12,E240726,3,8,8_V2,7.0,0,513.0,513.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [22]:
os.chdir(r"\\10.216.153.164\home\begas\27. AI활용 전기로 에너지 효율 최적화 기술개발\3차년도\OUT\python file")
agg_carbfeed_interval_stat.to_pickle(r"Tap_CarbTopFeed_투입횟수_2023.pkl")

In [31]:
agg_carbfeed_interval_stat[agg_carbfeed_interval_stat["HEAT_NO"] == "E238475"]

,Month,HEAT_NO,ActualCHRG_PX3,ActFceTap_Px3,ActFceTap_tag,Tap_sort_key,level_6,CarbTopFeed_min,CarbTopFeed_max,cnt_input,input_range,input_value_0,input_value_1,input_value_2,input_value_3,input_value_4,input_value_5,input_value_6,input_value_7,input_value_10,input_value_11,input_value_12,input_value_13,input_value_14,input_value_15,input_value_16,input_value_17,input_value_18,input_value_19,input_value_8,input_value_9,input_value_20,input_value_21,input_value_22,input_value_23,input_value_24,input_value_25,input_value_26,input_value_27,input_value_28,input_value_29,input_value_30,input_value_31,input_value_32,input_value_33,input_value_34,input_value_35,input_value_36,input_value_37,input_value_38,input_value_39,input_value_40,input_value_41,input_value_42,input_value_43,input_value_44,input_value_45,input_value_46,input_value_47,input_value_48,input_value_49,input_value_50,input_value_51,input_value_52,input_value_53,input_value_54,input_value_55,input_value_56,input_value_57,input_value_58,input_value_59,input_value_60,input_value_61,input_value_62,input_value_63,input_value_64,input_value_65,input_value_66,input_value_67,input_value_68,input_value_69,input_value_70,input_value_71,charge_input_count
69565,7,E238475,1,7,7_V1,1.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
69566,7,E238475,1,8,8_V1,2.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
69567,7,E238475,1,9,9_V1,3.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
69569,7,E238475,1,10,10_V1,4.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
69570,7,E238475,1,11,11_V1,5.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
69571,7,E238475,1,12,12_V1,6.0,0,0.0,516.0,1,516.0,0.0,516.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
69568,7,E238475,1,9,9_V2,7.0,0,516.0,516.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
69572,7,E238475,2,7,7_V1,1.0,0,516.0,516.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
69573,7,E238475,2,8,8_V1,2.0,0,516.0,516.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [53]:
# 전력사용량, 산소사용량, 가탄사용량 // "PX3BurnerO2", "PX3LanceO2", "Px3ChargeCarbon", "PX3InjecCarbon"
# 최대값 기준으로 집계 후 구간 별 사용량 계산 
# Offgas 괴탄 : 
agg_MWH_interval_stat = fulldata.groupby(["Month", "HEAT_NO", "ActualCHRG_PX3", 
                                          "ActFceTap_Px3", "ActFceTap_tag", "Tap_sort_key"]).agg(
    
    
    #MWH_value = ("Charge_MWH", lambda x : max(x) - min(x))
    MWH_max = ("Charge_MWH", lambda x : max(x)),
    MWH_min = ("Charge_MWH", lambda x : min(x)),
    
    # 
    burner_max = ("PX3BurnerO2", lambda x : max(x)),
    burner_min = ("PX3BurnerO2", lambda x : min(x)),
    
    # 
    lance_max = ("PX3LanceO2", lambda x : max(x)),
    lance_min = ("PX3LanceO2", lambda x : min(x)),
    
    # 
    chrgc_max = ("Px3ChargeCarbon", lambda x : max(x)),
    chrgc_min = ("Px3ChargeCarbon", lambda x : min(x)),
    
    # 
    injtc_max = ("PX3InjecCarbon", lambda x : max(x)),
    injtc_min = ("PX3InjecCarbon", lambda x : min(x))
    
    

).reset_index()

# save
agg_MWH_interval_stat00 = agg_MWH_interval_stat


def fn_calculate_heat_chrmwh(data) :
    
    # reorder
    temp_dt = data.sort_values("Tap_sort_key", ascending = True)
    
    # 
    temp_dt["MWH_value"] = temp_dt["MWH_max"] - temp_dt["MWH_max"].shift(1)
    temp_dt["MWH_value"] = np.where(pd.isna(temp_dt["MWH_value"]), 
                                    temp_dt["MWH_max"] - temp_dt["MWH_min"],
                                    temp_dt["MWH_value"])    

    # 
    temp_dt["burner_value"] = temp_dt["burner_max"] - temp_dt["burner_max"].shift(1)
    temp_dt["burner_value"] = np.where(pd.isna(temp_dt["burner_value"]), 
                                    temp_dt["burner_max"] - temp_dt["burner_min"],
                                    temp_dt["burner_value"])    
    
    # 
    temp_dt["lance_value"] = temp_dt["lance_max"] - temp_dt["lance_max"].shift(1)
    temp_dt["lance_value"] = np.where(pd.isna(temp_dt["lance_value"]), 
                                    temp_dt["lance_max"] - temp_dt["lance_min"],
                                    temp_dt["lance_value"])    
    
    # 
    temp_dt["chrgc_value"] = temp_dt["chrgc_max"] - temp_dt["chrgc_max"].shift(1)
    temp_dt["chrgc_value"] = np.where(pd.isna(temp_dt["chrgc_value"]), 
                                    temp_dt["chrgc_max"] - temp_dt["chrgc_min"],
                                    temp_dt["chrgc_value"])    
    
    # 
    temp_dt["injtc_value"] = temp_dt["injtc_max"] - temp_dt["injtc_max"].shift(1)
    temp_dt["injtc_value"] = np.where(pd.isna(temp_dt["injtc_value"]), 
                                    temp_dt["injtc_max"] - temp_dt["injtc_min"],
                                    temp_dt["injtc_value"])    
    
    
    
    # 
    return_dt = temp_dt[["ActFceTap_Px3", "ActFceTap_tag", "Tap_sort_key", 
                         "MWH_value", "burner_value", "lance_value", "chrgc_value", "injtc_value"]]
    
    return return_dt
    
;

# 
agg_MWH_interval_stat = agg_MWH_interval_stat.groupby(["Month", "HEAT_NO", "ActualCHRG_PX3"]).apply(fn_calculate_heat_chrmwh).reset_index()

agg_MWH_interval_stat.rename(columns = {"ActFceTap_tag" : "전압번호순서_Tag", "Tap_sort_key" : "전압인가순서"}, inplace = True)
agg_MWH_interval_stat


,Month,HEAT_NO,ActualCHRG_PX3,level_3,ActFceTap_Px3,전압번호순서_Tag,전압인가순서,MWH_value,burner_value,lance_value,chrgc_value,injtc_value
0,1,E235375,1,0,7,7_V1,1.0,0.999634,6.0,0.0,0.0,0.0
1,1,E235375,1,1,8,8_V1,2.0,0.000000,0.0,0.0,0.0,0.0
2,1,E235375,1,2,9,9_V1,3.0,0.100586,1.0,0.0,0.0,0.0
3,1,E235375,1,3,10,10_V1,4.0,0.399414,2.0,0.0,0.0,0.0
4,1,E235375,1,5,11,11_V1,5.0,0.000000,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
120044,12,E240726,3,120047,10,10_V2,5.0,5.097656,29.0,315.0,0.0,32.0
120045,12,E240726,3,120045,9,9_V2,6.0,0.101562,1.0,6.0,0.0,0.0
120046,12,E240726,3,120043,8,8_V2,7.0,0.101562,0.0,6.0,0.0,0.0
120047,12,E240726,3,120041,7,7_V1,8.0,0.101562,1.0,7.0,0.0,0.0


In [54]:
# 전압인가 순서 별 시간 확인 
agg_rap_time_interval_stat = fulldata.groupby(["Month", "HEAT_NO", "ActualCHRG_PX3", 
                                               "ActFceTap_Px3", "ActFceTap_tag", "Tap_sort_key"]).agg(
    On_time = ("Time", lambda x : (max(x) - min(x)).total_seconds()) 
    
).reset_index()


# 각 HEAT & Charge 별 최대 Tap 변경 횟수 확인 
agg_rap_time_interval_stat["cnt_change_tap"] = agg_rap_time_interval_stat.groupby(["Month", "HEAT_NO", "ActualCHRG_PX3"])["Tap_sort_key"].transform(lambda x : len(x))

# 각 HEAT & Charge 별 Tap 변동 패턴화 (ActFceTap_Px3 group concat)
# sorting 
agg_rap_time_interval_stat.sort_values(by = ["Month", "HEAT_NO", "ActualCHRG_PX3", "Tap_sort_key"], ascending = True, inplace = True)
agg_rap_time_interval_stat["patt_change_tap"] = agg_rap_time_interval_stat.groupby(["Month", "HEAT_NO", "ActualCHRG_PX3"])["ActFceTap_Px3"].transform(lambda x : "_".join(list(x.astype("string")))) 

# 
agg_rap_time_interval_stat.rename(columns = {"ActFceTap_tag" : "전압번호순서_Tag", "Tap_sort_key" : "전압인가순서"}, inplace = True)
agg_rap_time_interval_stat

,Month,HEAT_NO,ActualCHRG_PX3,ActFceTap_Px3,전압번호순서_Tag,전압인가순서,On_time,cnt_change_tap,patt_change_tap
0,1,E235375,1,7,7_V1,1.0,76.0,8,7_8_9_10_11_12_11_10
1,1,E235375,1,8,8_V1,2.0,3.0,8,7_8_9_10_11_12_11_10
2,1,E235375,1,9,9_V1,3.0,3.0,8,7_8_9_10_11_12_11_10
3,1,E235375,1,10,10_V1,4.0,27.0,8,7_8_9_10_11_12_11_10
5,1,E235375,1,11,11_V1,5.0,3.0,8,7_8_9_10_11_12_11_10
...,...,...,...,...,...,...,...,...,...
120047,12,E240726,3,10,10_V2,5.0,406.0,9,8_9_10_11_10_9_8_7_6
120045,12,E240726,3,9,9_V2,6.0,3.0,9,8_9_10_11_10_9_8_7_6
120043,12,E240726,3,8,8_V2,7.0,3.0,9,8_9_10_11_10_9_8_7_6
120041,12,E240726,3,7,7_V1,8.0,3.0,9,8_9_10_11_10_9_8_7_6


In [55]:
# arc 전극 집계 대상 항목 
add_etc_cols_arc = ['Current1', 'Current2', 'Current3', 
                    'Voltage1', 'Voltage2', 'Voltage3',
                    "ZFeedback1", "ZFeedback2", "ZFeedback3", 
                    "Res1", "Res2", "Res3", 
                    "Reactance1", "Reactance2", "Reactance3", 
                    "ArcLength1", "ArcLength2", "ArcLength3",
                    "MVA1", "MVA2", "MVA3", 
                    "MVATot", "MWTot", "MVARTot", "PFTot", 
                    "MW1", "MW2", "MW3", 
                    "MVAR1", "MVAR2", "MVAR3", 
                    "PF1", "PF2", "PF3", 
                    "RWI1", "RWI2", "RWI3", 
                    "ArcEfficiency1", "ArcEfficiency2", "ArcEfficiency3", 
                    "I2H1", "I2H2", "I2H3", 
                    "VoltageLineToLine12", "VoltageLineToLine23", "VoltageLineToLine31",
                    'SF1','SF2','SF3', 
                    "VoltSF1", "VoltSF2", "VoltSF3", 
                    "ExtSmartFactor1", "ExtSmartFactor2", "ExtSmartFactor3",
                    "B5_CarbonFlow", "B1_CarbonFlow", "CarbTopFeed_Cons", 
                    "B5_O2MainFlow", "B1_O2MainFlow", "B2_O2MainFlow", "B4_O2MainFlow", 
                    "B1_O2ShroudFlow", "B2_O2ShroudFlow", "B4_O2ShroudFlow",
                    "SFCE_CO2_Pct", "SFCE_CO_Pct", "SFCE_H2O_Pct", "SFCE_Offgas_MassFlow",
                    "NormH2OPctFlow", "NormCO2PctFlow", "NormCOPctFlow", "COCO2Ratio_Clus", 
                    "PCFactor"]  

In [56]:
# aggregations 
agg_arc_elct_interval_stat = fulldata.groupby(["Month", "HEAT_NO", "ActualCHRG_PX3", 
                                               "ActFceTap_Px3", "ActFceTap_tag", "Tap_sort_key"])[add_etc_cols_arc].agg(
    
    ["min", "median", "max", "mean", "std"]
    
).reset_index()

# 
agg_arc_elct_interval_stat.columns = ["_".join(pair) for pair in agg_arc_elct_interval_stat.columns]

# 
agg_arc_elct_interval_stat.rename(columns = {"Month_" : "Month",
                                        "HEAT_NO_" : "HEAT_NO",
                                        "ActualCHRG_PX3_" : "ActualCHRG_PX3",
                                        "ActFceTap_tag_" : "ActFceTap_tag",
                                        "ActFceTap_Px3_" : "ActFceTap_Px3",
                                        "Tap_sort_key_" : "Tap_sort_key"}, inplace = True)

agg_arc_elct_interval_stat.sort_values(["Month", "ActualCHRG_PX3", "Tap_sort_key", "ActFceTap_Px3"], inplace = True)
agg_arc_elct_interval_stat


,Month,HEAT_NO,ActualCHRG_PX3,ActFceTap_Px3,ActFceTap_tag,Tap_sort_key,Current1_min,Current1_median,Current1_max,Current1_mean,Current1_std,Current2_min,Current2_median,Current2_max,Current2_mean,Current2_std,Current3_min,Current3_median,Current3_max,Current3_mean,Current3_std,Voltage1_min,Voltage1_median,Voltage1_max,Voltage1_mean,Voltage1_std,Voltage2_min,Voltage2_median,Voltage2_max,Voltage2_mean,Voltage2_std,Voltage3_min,Voltage3_median,Voltage3_max,Voltage3_mean,Voltage3_std,ZFeedback1_min,ZFeedback1_median,ZFeedback1_max,ZFeedback1_mean,ZFeedback1_std,ZFeedback2_min,ZFeedback2_median,ZFeedback2_max,ZFeedback2_mean,ZFeedback2_std,ZFeedback3_min,ZFeedback3_median,ZFeedback3_max,ZFeedback3_mean,ZFeedback3_std,Res1_min,Res1_median,Res1_max,Res1_mean,Res1_std,Res2_min,Res2_median,Res2_max,Res2_mean,...,B1_O2ShroudFlow_min,B1_O2ShroudFlow_median,B1_O2ShroudFlow_max,B1_O2ShroudFlow_mean,B1_O2ShroudFlow_std,B2_O2ShroudFlow_min,B2_O2ShroudFlow_median,B2_O2ShroudFlow_max,B2_O2ShroudFlow_mean,B2_O2ShroudFlow_std,B4_O2ShroudFlow_min,B4_O2ShroudFlow_median,B4_O2ShroudFlow_max,B4_O2ShroudFlow_mean,B4_O2ShroudFlow_std,SFCE_CO2_Pct_min,SFCE_CO2_Pct_median,SFCE_CO2_Pct_max,SFCE_CO2_Pct_mean,SFCE_CO2_Pct_std,SFCE_CO_Pct_min,SFCE_CO_Pct_median,SFCE_CO_Pct_max,SFCE_CO_Pct_mean,SFCE_CO_Pct_std,SFCE_H2O_Pct_min,SFCE_H2O_Pct_median,SFCE_H2O_Pct_max,SFCE_H2O_Pct_mean,SFCE_H2O_Pct_std,SFCE_Offgas_MassFlow_min,SFCE_Offgas_MassFlow_median,SFCE_Offgas_MassFlow_max,SFCE_Offgas_MassFlow_mean,SFCE_Offgas_MassFlow_std,NormH2OPctFlow_min,NormH2OPctFlow_median,NormH2OPctFlow_max,NormH2OPctFlow_mean,NormH2OPctFlow_std,NormCO2PctFlow_min,NormCO2PctFlow_median,NormCO2PctFlow_max,NormCO2PctFlow_mean,NormCO2PctFlow_std,NormCOPctFlow_min,NormCOPctFlow_median,NormCOPctFlow_max,NormCOPctFlow_mean,NormCOPctFlow_std,COCO2Ratio_Clus_min,COCO2Ratio_Clus_median,COCO2Ratio_Clus_max,COCO2Ratio_Clus_mean,COCO2Ratio_Clus_std,PCFactor_min,PCFactor_median,PCFactor_max,PCFactor_mean,PCFactor_std
0,1,E235375,1,7,7_V1,1.0,20.187500,50.12500,67.50000,48.18750,12.146808,26.671875,48.12500,66.50000,47.87500,8.407855,20.593750,45.12500,61.46875,45.46875,9.159188,226.000000,381.00,537.00,381.75,74.797800,194.875000,385.25,492.00,374.75,63.744388,295.750000,398.50,547.50,400.00,50.683236,3.349609,7.531250,26.609375,9.007812,4.844627,3.640625,7.992188,14.343750,8.109375,2.173936,4.808594,8.812500,24.937500,9.593750,4.487206,1.919922,6.654297,19.984375,7.591421,3.937101,2.169922,6.734375,11.742188,6.747521,...,0,0.0,27,4.307692,8.577968,0,0.0,17,3.346154,5.321220,0,0.0,16,3.230769,4.885142,3.199219,3.549805,4.601562,3.673227,0.380097,0.0,0.0,20000.0,6153.846191,9413.574487,5.300781,6.101562,7.500000,6.180890,0.608617,0.0,0.0,0.0,0.0,0.0,25.234375,31.218750,33.343750,29.780048,2.792229,15.156250,18.140625,19.437500,17.666166,1.344027,0.0,0.000000,9.171875,2.555739,3.923539,1.802734,14.546875,44.75000,17.109375,12.661712,0.629737,1.000000,1.0,0.894571,0.161795
24,1,E235377,1,7,7_V1,1.0,14.437500,45.90625,87.18750,52.43750,18.699164,28.265625,59.90625,78.00000,55.18750,13.685705,35.218750,49.12500,75.87500,52.09375,11.960881,224.750000,367.75,619.50,372.50,105.783645,207.500000,399.75,617.00,389.25,109.378282,210.000000,388.75,556.50,380.00,84.473812,2.949219,7.859375,42.875000,9.234375,7.981027,2.720703,6.480469,16.937500,7.871094,3.898216,3.150391,7.671875,13.140625,7.804688,2.729669,1.230469,5.769531,15.781250,6.138998,3.888604,1.200195,5.230469,10.570312,5.638672,...,0,0.0,14,2.185185,4.394376,0,0.0,22,4.333333,6.456958,0,0.0,19,2.666667,5.356520,4.300781,5.000000,5.800781,5.018663,0.433180,0.0,0.0,20000.0,5925.925781,9306.432539,7.898438,9.203125,10.601562,9.233507,0.746751,0.0,0.0,0.0,0.0,0.0,27.156250,32.125000,34.406250,31.254051,2.162080,15.000000,17.187500,18.375000,16.969908,1.056826,0.0,0.000000,6.621094,1.777488,2.799984,1.082031,10.273438,39.68750,14.835938,12.897817,0.697013,1.000000,1.0,0.918490,0.128238
48,1,E235378,1,7,7_V1,1.0,25.828125,46.56250,96.00000,50.28125,16.2

In [57]:
# 설계치
spec_table_2022 = pd.DataFrame({"ActFceTap_Px3" : [6, 7, 8, 9, 10, 11, 12],
                                "Vol_spec_values" : np.arange(700, 910, 30),
                                "current_spec_min" : [55, 53, 50.5, 49, 48, 45, 42],
                                "current_spec_max" : [56, 53, 50.5, 49, 48, 45, 43]})

In [58]:
agg_arc_elct_copy = agg_arc_elct_interval_stat.copy()


# 
agg_arc_elct_copy = pd.merge(agg_arc_elct_copy, 
                             spec_table_2022, on = "ActFceTap_Px3", how = "left") 

# 
agg_arc_elct_copy = pd.merge(agg_arc_elct_copy, 
                             KWH_T_table[["Month", "HEAT_NO", "ActualCHRG_PX3", "KWH_T", "MES_ton", "input_chr_MWH"]].drop_duplicates(),             
                             on = ["Month", "HEAT_NO", "ActualCHRG_PX3"])

#
agg_arc_elct_copy.rename(columns = {"Tap_sort_key" : "전압인가순서",
                                    "ActFceTap_tag" : "전압번호순서_Tag",
                                    "KWH_T" : '장입전력원단위',
                                    "MES_ton" : "장입량",
                                    "input_chr_MWH" : "charge_전력사용량"}, inplace = True)

# copy
agg_check_taporders = agg_arc_elct_copy.copy()

# 
agg_check_taporders.sort_values(["HEAT_NO", "ActualCHRG_PX3", "전압인가순서"], inplace = True)
agg_check_taporders["check_order"] = agg_check_taporders.groupby(["HEAT_NO", "ActualCHRG_PX3"])["ActFceTap_Px3"].transform(lambda x : (x - x.shift(1)).fillna(9999999))
# agg_check_taporders["tag_order"] = [int(x[-1]) for x in agg_check_taporders["전압번호순서_Tag"]]
# agg_check_taporders["UD_tag"] = np.where(agg_check_taporders["tag_order"]%2 != 0, "Up", "Down")
agg_check_taporders["UD_tag"] = np.where(agg_check_taporders["check_order"] > 0, "Up", "Down")
agg_check_taporders["Tap_tag"] = ["Tap" + str(x) for x in agg_check_taporders["ActFceTap_Px3"]]


# 전력사용량, Ontime data 병합하기 
aggr_allcolumns_dt = reduce(lambda x, y : pd.merge(x, y, on = ["Month", "HEAT_NO", "ActualCHRG_PX3", 
                                                               "ActFceTap_Px3", "전압번호순서_Tag", "전압인가순서"]), 
                            [agg_check_taporders, agg_MWH_interval_stat, agg_rap_time_interval_stat])

aggr_allcolumns_dt.sort_values(["Month", "HEAT_NO", "ActualCHRG_PX3", "전압인가순서", "ActFceTap_Px3"], inplace = True)
aggr_allcolumns_dt

,Month,HEAT_NO,ActualCHRG_PX3,ActFceTap_Px3,전압번호순서_Tag,전압인가순서,Current1_min,Current1_median,Current1_max,Current1_mean,Current1_std,Current2_min,Current2_median,Current2_max,Current2_mean,Current2_std,Current3_min,Current3_median,Current3_max,Current3_mean,Current3_std,Voltage1_min,Voltage1_median,Voltage1_max,Voltage1_mean,Voltage1_std,Voltage2_min,Voltage2_median,Voltage2_max,Voltage2_mean,Voltage2_std,Voltage3_min,Voltage3_median,Voltage3_max,Voltage3_mean,Voltage3_std,ZFeedback1_min,ZFeedback1_median,ZFeedback1_max,ZFeedback1_mean,ZFeedback1_std,ZFeedback2_min,ZFeedback2_median,ZFeedback2_max,ZFeedback2_mean,ZFeedback2_std,ZFeedback3_min,ZFeedback3_median,ZFeedback3_max,ZFeedback3_mean,ZFeedback3_std,Res1_min,Res1_median,Res1_max,Res1_mean,Res1_std,Res2_min,Res2_median,Res2_max,Res2_mean,...,SFCE_CO2_Pct_mean,SFCE_CO2_Pct_std,SFCE_CO_Pct_min,SFCE_CO_Pct_median,SFCE_CO_Pct_max,SFCE_CO_Pct_mean,SFCE_CO_Pct_std,SFCE_H2O_Pct_min,SFCE_H2O_Pct_median,SFCE_H2O_Pct_max,SFCE_H2O_Pct_mean,SFCE_H2O_Pct_std,SFCE_Offgas_MassFlow_min,SFCE_Offgas_MassFlow_median,SFCE_Offgas_MassFlow_max,SFCE_Offgas_MassFlow_mean,SFCE_Offgas_MassFlow_std,NormH2OPctFlow_min,NormH2OPctFlow_median,NormH2OPctFlow_max,NormH2OPctFlow_mean,NormH2OPctFlow_std,NormCO2PctFlow_min,NormCO2PctFlow_median,NormCO2PctFlow_max,NormCO2PctFlow_mean,NormCO2PctFlow_std,NormCOPctFlow_min,NormCOPctFlow_median,NormCOPctFlow_max,NormCOPctFlow_mean,NormCOPctFlow_std,COCO2Ratio_Clus_min,COCO2Ratio_Clus_median,COCO2Ratio_Clus_max,COCO2Ratio_Clus_mean,COCO2Ratio_Clus_std,PCFactor_min,PCFactor_median,PCFactor_max,PCFactor_mean,PCFactor_std,Vol_spec_values,current_spec_min,current_spec_max,장입전력원단위,장입량,charge_전력사용량,check_order,UD_tag,Tap_tag,level_3,MWH_value,burner_value,lance_value,chrgc_value,injtc_value,On_time,cnt_change_tap,patt_change_tap
0,1,E235375,1,7,7_V1,1.0,20.187500,50.12500,67.500000,48.18750,12.146808,26.671875,48.12500,66.500000,47.87500,8.407855,20.593750,45.12500,61.46875,45.46875,9.159188,226.000000,381.0,537.00,381.75,74.797800,194.875000,385.25,492.00,374.75,63.744388,295.750000,398.50,547.50,400.000,50.683236,3.349609,7.531250,26.609375,9.007812,4.844627,3.640625,7.992188,14.343750,8.109375,2.173936,4.808594,8.812500,24.937500,9.593750,4.487206,1.919922,6.654297,19.984375,7.591421,3.937101,2.169922,6.734375,11.742188,6.747521,...,3.673227,0.380097,0.0,0.0,20000.0,6153.846191,9413.574487,5.300781,6.101562,7.500000,6.180890,0.608617,0.0,0.0,0.0,0.0,0.0,25.234375,31.218750,33.343750,29.780048,2.792229,15.156250,18.140625,19.437500,17.666166,1.344027,0.0,0.000000,9.171875,2.555739,3.923539,1.802734,14.546875,44.750000,17.109375,12.661712,0.629737,1.000000,1.0,0.894571,0.161795,730,53.0,53.0,307.543993,51.7,15.900024,9999999.0,Up,Tap7,0,0.999634,6.0,0.0,0.0,0.0,76.0,8,7_8_9_10_11_12_11_10
1,1,E235375,1,8,8_V1,2.0,17.937500,20.96875,23.984375,20.96875,4.275786,25.984375,29.81250,33.656250,29.81250,5.424835,28.953125,30.40625,31.84375,30.40625,2.043981,537.500000,584.0,631.00,584.00,66.114484,285.500000,402.25,519.00,402.25,165.109433,201.000000,344.00,487.00,344.000,202.232539,22.406250,28.812500,35.218750,28.812500,9.059806,8.492188,14.218750,19.953125,14.218750,8.104107,6.308594,11.562500,16.812500,11.562500,7.427383,18.062500,19.242188,20.421875,19.242188,1.668330,7.539062,11.472656,15.406250,11.472656,...,3.400391,0.000000,0.0,0.0,0.0,0.000000,0.000000,6.699219,6.798828,6.898438,6.798828,0.140869,0.0,0.0,0.0,0.0,0.0,33.156250,33.328125,33.500000,33.328125,0.243068,16.500000,16.664062,16.828125,16.664062,0.232019,0.0,0.000000,0.000000,0.000000,0.000000,1.208008,1.306641,1.406250,1.306641,0.140178,1.000000,1.000000,1.0,1.000000,0.000000,760,50.5,50.5,307.543993,51.7,15.900024,1.0,Up,Tap8,1,0.000000,0.0,0.0,0.0,0.0,3.0,8,7_8_9_10_11_12_11_10
2,1,E235375,1,9,9_V1,3.0,23.234375,30.21875,37.218750,30.21875,9.888446,23.875000,38.00000,52.093750,38.00000,19.953669,32.656250,38.56250,44.50000,38.56250,8.374796,510.000000,528.0,546.00,528.00,25.455844,262.750000,360.25,457.75,

In [59]:
# total file : 2023
# os.chdir(r"D:\국책과제_2차년도\DAT\전압인가순서적용\arc요약")
os.chdir(r"\\10.216.153.164\home\begas\27. AI활용 전기로 에너지 효율 최적화 기술개발\2차년도\OUT\csv\전압인가순서적용\arc요약")
aggr_allcolumns_dt.to_csv(r"2023_HEAT별 arc요약_전압인가순서적용.csv", index = False, encoding = "cp949")
agg_vsums_heat_stat.to_csv(r"2023_HEAT별 arc요약_SF총합.csv", index = False, encoding = "cp949")

In [28]:
os.chdir(r"D:\국책과제_2차년도\DAT\전압인가순서적용\arc요약")
aggr_allcolumns_dt[aggr_allcolumns_dt["Month"].between(1, 3)].to_csv(r"2023_HEAT별 arc요약_전압인가순서적용_Q1.csv", index = False, encoding = "cp949")
aggr_allcolumns_dt[aggr_allcolumns_dt["Month"].between(4, 6)].to_csv(r"2023_HEAT별 arc요약_전압인가순서적용_Q2.csv", index = False, encoding = "cp949")
aggr_allcolumns_dt[aggr_allcolumns_dt["Month"].between(7, 9)].to_csv(r"2023_HEAT별 arc요약_전압인가순서적용_Q3.csv", index = False, encoding = "cp949")
aggr_allcolumns_dt[aggr_allcolumns_dt["Month"].between(10, 12)].to_csv(r"2023_HEAT별 arc요약_전압인가순서적용_Q4.csv", index = False, encoding = "cp949")